In [1]:
import concurrent.futures
import io
import json
import os
import sys
import threading
import traceback

import nbformat
from dotenv import find_dotenv, load_dotenv
from fuzzywuzzy import fuzz
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from openai import OpenAI
from utils import get_delivered_df, DATA_DIR, PROJECT_ROOT

from src.llm_reviewer.constants import Roles
from src.llm_reviewer.llm_api import LLMAPIFactory, make_llm_request


def download_file(service_account_file, file_id, revision_id=None):
    # Authenticate with the service account
    credentials = service_account.Credentials.from_service_account_file(
        service_account_file, scopes=["https://www.googleapis.com/auth/drive"]
    )
    service = build("drive", "v3", credentials=credentials)

    # Request to download the file, optionally specifying a revision
    if revision_id is not None:
        request = service.revisions().get_media(fileId=file_id, revisionId=revision_id)
    else:
        request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    # Download the file
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print("Download progress: %d%%." % int(status.progress() * 100))

    # Move the buffer's pointer to the beginning
    fh.seek(0)

    # Read the notebook content as JSON
    payload = json.load(fh)
    return payload


from src.gdrive_api.folder_upload import upload_folder
from utils import service_account_path



MAIN_DIR = 'https://drive.google.com/drive/folders/1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR'



/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
import utils
from utils import service_account_path
from src.sheets_utils import download_sheet_as_df

from utils import get_delivered_df

batch_5 = get_delivered_df([5])
batch_5 = batch_5.assign(jsonl_file_id=batch_5['jsonl_link'].apply(lambda x: x.split('/file/d/')[1].split('/')[0]))

sheet_id = '1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4'
redo_df = download_sheet_as_df(service_account_path, sheet_id, 'gpt_flags_2')
print('Total:', len(redo_df))
done_df = redo_df[redo_df['status'] == 'Done']
done_df = done_df.merge(batch_5[['task_link', 'jsonl_file_id']], on='task_link', how='left')
done_df = done_df.assign(file_id=done_df['task_link'].apply(lambda x: x.split('/')[-1]))
print('DONE:', len(done_df))
#done_df


Total: 59
DONE: 53


In [ ]:
from jsonl_dump import *



def download_parse_delivered_into_jsonl(
    batch_df, max_workers=20, no_work=False
):
    delivered_df = batch_df  # Assuming batch_id 4 is required
    if not no_work:
        parsed_conversations = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            file_ids = [link.split("/")[-1] for link in delivered_df["task_link"]]
            parsed_conversations = list(
                executor.map(
                    download_and_parse_notebook,
                    [service_account_path] * len(file_ids),
                    file_ids,
                )
            )

        for conversation in parsed_conversations:
            if conversation is None:
                continue
            colab_link = conversation["colab_link"]
            batch_id = None
            for b_id, task_link in zip(
                delivered_df["batch_id"], delivered_df["task_link"]
            ):
                if colab_link.endswith(task_link.split("/")[-1]):
                    batch_id = b_id
                    break
            if batch_id is None:
                raise
            batch_name = f"batch_{batch_id}"
            batch_folder = f"{DATA_DIR}jsonl_conversations/{batch_name}/"
            if not os.path.exists(batch_folder):
                os.makedirs(batch_folder)
            drive_id = conversation["id"]
            with open(f"{batch_folder}{drive_id}.json", "w") as f:
                f.write(json.dumps({"batch_id": batch_id, **conversation}))
    else:
        parsed_conversations = []
        for batch_id in batch_ids:
            batch_name = f"batch_{batch_id}"
            batch_folder = f"{DATA_DIR}jsonl_conversations/{batch_name}/"
            if os.path.exists(batch_folder):
                for file_name in os.listdir(batch_folder):
                    if file_name.endswith(".json"):
                        with open(f"{batch_folder}{file_name}", "r") as f:
                            conversation = json.load(f)
                            parsed_conversations.append(conversation)
            else:
                raise Exception(f"Batch folder for {batch_name} not found")

    return {"delivered_df": delivered_df, "conversations": parsed_conversations}

batch_5['batch_id'] = 66

batch_df = batch_5[batch_5['task_link'].isin(done_df['task_link'])]
data = download_parse_delivered_into_jsonl(batch_df)
print('Downloaded jsonl', len(data['conversations']))

import pandas as pd
convos = data['conversations']
# Create a DataFrame with only 'colab_link' and 'messages' from the conversations list
convos_df = pd.DataFrame(convos, columns=['colab_link', 'messages'])
merged_df = pd.merge(done_df, convos_df, left_on='task_link', right_on='colab_link')

merged_df['jsonl_raw'] = merged_df['jsonl_file_id'].apply(lambda x: download_file(service_account_path, x))

for index, row in merged_df.iterrows():
    row['jsonl_raw']['messages'] = row['messages']

In [ ]:
merged_df['colab_file_id'] = merged_df['task_link'].apply(lambda x: x.split('drive/')[1])



from src.gdrive_api.folder_upload import upload_file
from src.gdrive_api.auth import build_service

for index, row in merged_df.iterrows():
    print(f"Processing {index+1}/{len(merged_df)}")
    json_file_name_only_name = row['colab_file_id'].replace(' ', '') + '.json'
    os.makedirs(DATA_DIR + 'FIXED5/', exist_ok=True)
    jsonl_filename = DATA_DIR + 'FIXED5/' + json_file_name_only_name
    with open(jsonl_filename, 'w') as jsonl_file:
        json.dump(row['jsonl_raw'], jsonl_file)
    upload_file(build_service(service_account_path), jsonl_filename, '1pEC7hlH3DTMUrkEHeDZduG7AyZf2lSRR', force_replace=True)